In [1]:
from process import *
from process_theory import *

# Defining a process theory

A process theory defines equations on processes. Processes are built up from atoms and process-operators. Atoms are divided into constants, variables, recursion-variables, and actions. A theory is an element of the `Theory` class. Its constructor takes zero of more keyword arguments. The arguments `conts` and `variables` give the lists of constants and variables of the theory. The remaining arguments are supposed to provide the axioms, i.e., process-equalities, of the theory. The following example shows how this works. 

TODO: add recursion-variables and actions!

In [2]:
blocked, ready = consts('blocked, ready')
x, y, z = vars('x, y, z')

# (Python) variable TH0 is assigned a process-theory
TH0 = Theory(
    AX1 = blocked * x == blocked,
    AX2 = ready * x == x,
    AX3 = x * ready == x,
    AX4 = x + blocked == x,
    AX5 = x + y == y + x,
    AX6 = x + x == x,
    AX7 = (x + y) + x == x + (y + z),
    AX8 = (x * y) * z == x * (y * z),
    AX9 = (x + y) * z == x * z + y * z,
    AX10 = Encap(blocked, x) == x,
    AX11 = Encap(ready, x) == x,
    # AX12 = Encap(x, blocked) == blocked,
    AX13 = Encap(x, ready) == ready,
    AX14 = Encap(x + y, z) == Encap(x, Encap(y, z)),
    AX15 = Encap(x * y, z) == Encap(x, Encap(y, z)),

)

The variables stand for arbitrary processes. Therefore, an axiom `AX1 = blocked * x == blocked` means that for all processes `x`, `blocked * x` is equal to `blocked`. The `*` process-operator is sequential composition. The meaning of `x * y` is the behaviour of `x` followed by the behaviour of `y`. If `x` is `blocked`, then `x * y` will never start the behaviour of `y`.

> The expressions in `Theory`s are normal Python expressions. Therefore, atoms and variables follow the syntax of Python variables and process-operators are limited to the Python operators defined on `CoreProcess`es: `+`, `*`, and `|`.

In [3]:
# attribute-access can be used for the theory's axioms
print(TH0.AX1)
print(TH0.AX6)

# and axiom has a left-hand side and a right-hand side, which
# can be accessed by lhs and rhs, respectively.

print(f'AX1 lhs: {TH0.AX1.lhs}, AX1 rhs: {TH0.AX1.rhs}')
print(f'AX6 lhs: {TH0.AX6.lhs}, AX1 rhs: {TH0.AX6.rhs}')

assert blocked in TH0.constants()
assert ready in TH0.constants()

for v in [x,y,z]:
    assert v in TH0.variables()
 

blocked * x == blocked
x + x == x
AX1 lhs: blocked * x, AX1 rhs: blocked
AX6 lhs: x + x, AX1 rhs: x


## Actions

Process theories can be used to model system behaviour. The smallest unit of system behaviour is called an *action*. In process theories, actions are assumed to be provided by the user. The function `atomic_actions()` is similar to the `atoms()` and `vars()` functions above, but instead of defining atoms or variables, it defines actions. The intention is that these actions capture the smallest units of behaviour of the system.

In [4]:
# define three actions
a,b,c = atomic_actions('a, b, c')

# create a process p built up from the actions
p = a * b * (a + b) * c

In [5]:
print(p)

a * b * (a + b) * c


`p` is a process that starts with action `a`, then `b` and then either an `a` or a `b` and finally a `c`. Another way to model this behaviour is by a process that starts with action `a`, then  `b`, and then `a`-followed-by-`c` or `b`-followed-by-`c`, as defined by process `q` below.  

In [6]:
q = a * b * (a * c + b * c)

# Proofs

Process `p` and `q` model the same behaviour. However, their definitions are not exactly the same. We can prove `p == q` by using axioms `AX8` and `AX9` of theory `TH0`:

In [7]:
prf = EqProof(p)
prf += a * b * ((a + b) * c), TH0.AX8
prf += q, TH0.AX9

The `prf` represents an equational proof (on processes). The typical style of equational proofs becomes visible if we print the proof:

In [8]:
print(prf)

  a * b * (a + b) * c
= {x * y * z == x * (y * z)}
  a * b * ((a + b) * c)
= {(x + y) * z == x * z + y * z}
  a * b * (a * c + b * c)


## Constructing Equational Proofs

The constructor `EqProof()` takes the initial process term as an argument. New process terms can be added to the proof by the `_ += _` in-place-add operator (Python's `__iadd__()` function). It takes the current proof, the target / next process term and zero or more "justifications" as arguments:

    prf += pnext, J1, ..., Jn

The in-place-add operation succeeds if `prf` can be extended to `pnext` by applying the equations listed in justifications `J1`..`Jn`.

> Note: The `_ += _` operator to extend equational proofs uses Z3, see [Introduction | Online Z3 Guide](https://microsoft.github.io/z3guide/programming/Z3%20Python%20-%20Readonly/Introduction/), to verify that the given target can be proven. In addition to the given justifications `J1`..`Jn`, also Z3's built-in rules will be used in the verification. The built-in rules are mostly for numerical calculations.

In the example above, the proof is extended twice. First by applying `AX8` and then by applying `AX9`.

If `pnext` cannot be proven by `J1`..`Jn`, then an exception is raised. It might take quite long before this happens. The exception or the computation does not end (in which case there will be a lower-level exception indicating some kind of overflow or memory shortage).


## Shortening the proof

The two steps in the proof above can be combined into one by providing both axioms at the same time:

In [9]:
prf = EqProof(p)
prf += q, TH0.AX8, TH0.AX9
print(prf)

  a * b * (a + b) * c
= {x * y * z == x * (y * z), (x + y) * z == x * z + y * z}
  a * b * (a * c + b * c)


## Failing proofs

If you try to extend a proof, but the target cannot be derived from the proof's current final process term using the justifications given, the proof extension fails and an `EqProofException` is raised. For instance, the proof above fails if `AX9` is but `AX8` is not provided as a justification:

In [10]:
# The proof attempt below will be aborted because of the *proof timeout*. This means 
# that the Z3 solver cannot proof the step with the proof timeout.
# Normally the proof timeout is 2 minutes.
print(f"Proof timeout: {timeout()}s" )

# Now we set it to 2 seconds:
set_timeout(2)

print("The proof is:")
print(prf)

# Exception handler added, because the proof is expected to fail
try:
    prf = EqProof(p)
    prf += q, TH0.AX9
    assert False, "this shall not happen"
except ProofException as pe:
    print(pe)

# The proof was not extended:
print("The proof was not changed:")
print(prf)

Proof timeout: 120.0s
The proof is:
  a * b * (a + b) * c
= {x * y * z == x * (y * z), (x + y) * z == x * z + y * z}
  a * b * (a * c + b * c)


<ProofTimeoutError>: Cannot proof a * b * (a + b) * c == a * b * (a * c + b * c) within 2.0s from (x + y) * z == x * z + y * z
The proof was not changed:
  a * b * (a + b) * c


The proof also fails if `AX8` is but `AX9` is not provided as a justification:

In [11]:
# Exception handler added, because the proof is expected to fail
try:
    prf = EqProof(p)
    prf += q, TH0.AX8
    assert False, "this shall not happen"
except ProofException as pe:
    print(pe)

# The proof was not extended:
print("The proof was not changed:")
print(prf)

<ProofTimeoutError>: Cannot proof a * b * (a + b) * c == a * b * (a * c + b * c) within 2.0s from x * y * z == x * (y * z)
The proof was not changed:
  a * b * (a + b) * c


The proof fails if `AX1` is provided as a justification

In [12]:
# Exception handler added, because the proof is expected to fail
try:
    prf = EqProof(p)
    prf += q, TH0.AX1
    assert False, "this shall not happen"
except ProofException as pe:
    print(pe)

# The proof was not extended:
print("The proof was not changed:")
print(prf)

<ProofError>: Cannot proof a * b * (a + b) * c == a * b * (a * c + b * c) from blocked * x == blocked
The proof was not changed:
  a * b * (a + b) * c


Note that this is a `<ProofError>` instead of a `<ProofTimeoutError>` as before. So, here the Z3 solver determined within `timeout()` seconds that the step cannot be proven.